## All Subsets Regression


It is possible to run through all of the possible combinations of predictors and generate the metrics we are most interested in.  We'll present some code below for finding the best model for each number of predictors.

In [ ]:

import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# read in the data to dataframe called ames
ames = pd.read_csv("https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/Ames_house_prices.csv", na_values=['?'])
# replace the ? in the data with NaN for missing values
ames.replace([' ?'],np.nan)
# show information about the dataframe
ames.info()

Here we apply the best subset selection approach to the Ames Housing data. We wish to predict the Sale Price based upon a variety of features/predictors.

In [ ]:
#print(ames.columns)
ames_df=ames[['LotArea','OverallQual','YearBuilt','BsmtFinSF1','GrLivArea','YrSold','GarageArea','PoolArea']]

In [ ]:
X=ames_df
y=ames['SalePrice']

We can perform best subset selection by identifying the best model that contains a given number of predictors, where **best** is quantified using RMSE. We'll define a helper function to outputs the best set of variables for each model size.

In [ ]:
# create a function called processSubset to fit the models and return information about
# the model as well as the RMSE
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    #X=sm.add_constant(X[list(feature_set)])
    model = sm.OLS(y,sm.add_constant(X[list(feature_set)]))
    regr = model.fit()
    RMSE = np.sqrt(((regr.predict(sm.add_constant(X[list(feature_set)])) - y) ** 2).mean())
    return {"model":regr, "RMSE":RMSE}

In [ ]:
def getBest(k):
    # start tracking the time 
    tic = time.time()
    # create an array to put the results
    results = []
    
    # do all combinations of predictors 
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the smallest RMSE
    best_model = models.loc[models['RMSE'].argmin()]
    
    # stop tracking the time
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

This returns a `DataFrame` containing the best model that we generated, along with some extra information about the model. Now we want to call that function for each number of predictors $k$:

In [ ]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["RMSE", "model"])

tic = time.time()
for i in range(1,7):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Now we have one big `DataFrame` that contains the best models we've generated for each number of predictors along with their RMSE:

In [ ]:
models_best

If we want to access the details of each model, no problem! We can get a full rundown of a single model using the `summary()` function:

In [ ]:
# here is the best model with 2 predictors is counted as a predictor
print(models_best.loc[2, "model"].summary())

This output indicates that the best two-variable model
contains only `OverallQual` and `GrLivArea`. To save time, we only generated results
up to the best 6-variable model. You can use the functions we defined above to explore as many variables as are desired.

In [ ]:
# Show the best 8-variable model 
print(getBest(8)["model"].summary())

Rather than letting the results of our call to the `summary()` function print to the screen, we can access just the parts we need using the model's attributes. For example, if we want the $R^2$ value:

In [ ]:
models_best.loc[5, "model"].rsquared

Excellent! In addition to the verbose output we get when we print the summary to the screen, fitting the `OLM` also produced many other useful statistics such as adjusted $R^2$, AIC, and BIC. We can examine these to try to select the best overall model. Let's start by looking at $R^2$ across all our models:

In [ ]:
# Gets the second element from each row ('model') and pulls out its rsquared attribute
models_best.apply(lambda row: row[1].rsquared, axis=1)

As expected, the $r^2$ statistic increases monotonically as more variables are included.  This happens even if the added predictor has little value; that's part of the reason that we introduced $r^2_{adj}$.

Plotting RSS, adjusted $r^2$, AIC, and BIC for all of the models at once will help us decide which model to select. 

Akaike's Information Criterion (AIC) and Bayesian Information Criterion (BIC) are two other metrics for penalizing extra terms in a model.  While $r^2_{adj}$ works well for regression, AIC and BIC are more general for a larger class of models.  For both AIC and BIC we want the smallest values we can get. 

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

# Set up a 2x2 grid so we can look at 4 plots at once
plt.subplot(2, 2, 1)

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
plt.plot(models_best["RMSE"])
plt.xlabel('# Predictors')
plt.ylabel('RMSE')

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector

rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

plt.subplot(2, 2, 2)
plt.plot(rsquared_adj)
plt.plot(rsquared_adj.argmax()+1, rsquared_adj.max(), "or")
plt.xlabel('# Predictors')
plt.ylabel('adjusted rsquared')

# We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
aic = models_best.apply(lambda row: row[1].aic, axis=1)

plt.subplot(2, 2, 3)
plt.plot(aic)
plt.plot(aic.argmin()+1, aic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('AIC')

bic = models_best.apply(lambda row: row[1].bic, axis=1)

plt.subplot(2, 2, 4)
plt.plot(bic)
plt.plot(bic.argmin()+1, bic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('BIC')

Recall that in the second step of our selection process, we narrowed the field down to just one model on any $k<=p$ predictors. We see that according to BIC, the best performer is the model with 6 variables.  Again, no one measure is going to give us an entirely accurate picture... but they all agree that a model with 6 predictors is pretty good.

Note that we don't stop here.  The process for choosing a final model needs to include a further investigation of these top models as well as a look at the residuals.

### Penguins

In [ ]:
penguins = pd.read_csv("https://webpages.charlotte.edu/mschuck1/classes/DTSC2301/Data/penguins.csv", na_values=['NA'])
# remove rows with missing data
penguins.dropna(inplace=True)
penguins.head()

In [ ]:
# make indicator/dummy variables for species
# each species gets a different column
one_hot=pd.get_dummies(penguins['species'],dtype=int)
print(one_hot)
penguins=penguins.join(one_hot)
penguins.head()

In [ ]:
# interactions of bill_depth_mm with Species
penguins['billDepth_Adelie'] = penguins['bill_depth_mm']*penguins['Adelie']
penguins['billDepth_Chinstrap'] = penguins['bill_depth_mm']*penguins['Chinstrap']

# interactions of flipper_length_mm with Species
penguins['flipper_Adelie'] = penguins['flipper_length_mm']*penguins['Adelie']
penguins['flipper_Chinstrap'] = penguins['flipper_length_mm']*penguins['Chinstrap']

# interactions of bill_length_mm with Species
penguins['billLength_Adelie'] = penguins['bill_length_mm']*penguins['Adelie']
penguins['billLength_Chinstrap'] = penguins['bill_length_mm']*penguins['Chinstrap']

In [ ]:

X = penguins[['bill_depth_mm','bill_length_mm','flipper_length_mm','Adelie','Chinstrap','billDepth_Adelie',
              'billDepth_Chinstrap','billLength_Adelie','billLength_Chinstrap','flipper_Adelie','flipper_Chinstrap']]
y = penguins['body_mass_g']

In [ ]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["RMSE", "model"])

tic = time.time()
for i in range(1,11):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Looking at the best models and their $r^2_adj$ and how that changes with model size.

In [ ]:
models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

Looking at this output we can see that $r^2_{adj}$ rises until we have 7 predictors and then begins to drop.  Let's look at that model.

In [ ]:
# Show the best 7-predictor model, the one with the highest r^2 adjusted
print(getBest(7)["model"].summary())

Note the std err for _Chinstrap_ that is very large and suggests multicollinearity.

In [ ]:
# Show the best 5-predictor model, the one with the highest r^2 adjusted
print(getBest(5)["model"].summary())

In [ ]:
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

# Set up a 2x2 grid so we can look at 4 plots at once
plt.subplot(2, 2, 1)

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector
plt.plot(models_best["RMSE"])
plt.xlabel('# Predictors')
plt.ylabel('RMSE')

# We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
# The argmax() function can be used to identify the location of the maximum point of a vector

rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

plt.subplot(2, 2, 2)
plt.plot(rsquared_adj)
plt.plot(rsquared_adj.argmax()+1, rsquared_adj.max(), "or")
plt.xlabel('# Predictors')
plt.ylabel('adjusted rsquared')

# We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
aic = models_best.apply(lambda row: row[1].aic, axis=1)

plt.subplot(2, 2, 3)
plt.plot(aic)
plt.plot(aic.argmin()+1, aic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('AIC')

bic = models_best.apply(lambda row: row[1].bic, axis=1)

plt.subplot(2, 2, 4)
plt.plot(bic)
plt.plot(bic.argmin()+1, bic.min(), "or")
plt.xlabel('# Predictors')
plt.ylabel('BIC')

So while $r^2_{adj}$ suggested that we consider a model with 7 predictors, AIC and BIC preferred the best model with 5 predictors.  

#### Next Steps
In the model building process we would not be finished.  As we noted from the plot above our metrics are not in agreement.  We should investigate the models that have values near the highest for $r^2_{adj}$ or the lowest for AIC and BIC.  So we should look at the best models with 5 predictors and the best models with 7 predictors.  And likely the best model with 6 predictors.

There are also other considerations for our models.  Namely, does the regression meet the conditions for using this kind of model and how does the model perform out of sample, that is via cross-validation.  

Model building is not a linear process.  We get some models to consider from all subsets regression but we are not finished.  We should investigate further the attributes of the models and develop new models.  

As many of you found with Project 1, at some point you have to decide this is the best model that I have and go with it OR don't because it is not of sufficient quality.  (Hope you didn't do the latter on Project 1.)

Also this process is just for linear regression not for spline regression or local regression or 
GAMs.  

### Tasks

1. Read in the bluejay data.  For this analysis we'll only use the quantitative variables plus 'Sex' 

2. Create interactions for 'Sex' and each of the other quantitative variables

3. Find the best regression models of up to size 8 predicting 'Mass' and generate the plot of our model metrics vs the number of predictors.

4. Which size of model do $r^2_{adj}$, AIC and BIC recommend?

5. Get summaries for each of the models in Task 4 and investigate each model.  From just looking at the summaries which model would you investigate first and why?

6. Investigate the model you chose in Task 5 and determine if the linear regression conditions are met for that model.